In [24]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import hvplot.pandas
import geopandas as gpd
import holoviews as hv
import geoviews as gv
import pandas as pd
import requests
from pathlib import Path
from bokeh.models import HoverTool

# Import API key
from api_keys import geoapify_key

In [25]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,badger,64.8000,-147.5333,-8.07,88,100,0.00,US,1699712408
1,1,lone grove,34.1754,-97.2628,10.55,66,100,2.06,US,1699712300
2,2,port-aux-francais,-49.3500,70.2167,3.22,62,19,5.72,TF,1699712409
3,3,severo-yeniseyskiy,60.3725,93.0408,-10.24,93,100,1.34,RU,1699712410
4,4,yatou,37.1500,122.3833,3.07,75,63,3.46,CN,1699712410


In [81]:

# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(city_data_df, geometry=gpd.points_from_xy(city_data_df['Lng'], city_data_df['Lat']))
gdf['Sizer'] = gdf['Humidity'] ** 0.5

# Create a Points element with hover information
points = gv.Points(gdf, vdims=['Lng', 'Lat', 'City', 'Humidity', 'Sizer'])

# Set a reasonable range for color mapping
color_range = (gdf['Sizer'].min(), gdf['Sizer'].max())

# Create the map using the GeoViews map function
result_map = gv.tile_sources.OSM() * points.opts(
    size='Sizer',
    color='City',  
    cmap='Category10',
    colorbar=True,  
    clim=color_range,  
    tools=[HoverTool(tooltips=[('Lng', '@Lng'), ('Lat', '@Lat'), ('City', '@City'), ('Humidity', '@Humidity')])]
).opts(
    width=1000,
    height=750,
    legend_position='right',
    legend_offset=(0, 0),  
    alpha=0.7
)

# Display the result
result_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (Lng,Lat,City,Humidity,Sizer)

In [27]:
ideal_cities = []

for index, row in city_data_df.iterrows():
    if row['Max Temp'] > 21 and row['Max Temp'] <27 and row['Wind Speed'] <4.5 and row['Cloudiness'] ==0:
        ideal_cities.append(row)

ideal_cities = pd.DataFrame(ideal_cities)
ideal_cities


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
43,43,howrah,22.5892,88.3103,26.00,69,0,2.06,IN,1699712441
66,66,khunti,23.0833,85.2833,22.06,51,0,2.18,IN,1699712454
246,246,tanumah,27.1000,44.1333,26.04,18,0,2.68,SA,1699712570
265,265,cabo san lucas,22.8909,-109.9124,25.84,74,0,2.55,MX,1699712487
298,298,figuig (centre),32.1089,-1.2286,25.55,25,0,3.47,MA,1699712600
348,348,san martin,-33.0810,-68.4681,23.30,29,0,3.20,AR,1699712630
438,438,vallenar,-28.5708,-70.7581,23.45,33,0,2.16,CL,1699712687
460,460,bechar,31.6167,-2.2167,26.18,22,0,3.60,DZ,1699712702
548,548,dajal,29.5577,70.3762,22.94,34,0,1.01,PK,1699712769
604,604,urangan,-25.3000,152.9000,21.42,67,0,3.48,AU,1699712801


In [28]:
hotel_df = ideal_cities[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
hotel_df['Hotel'] = ''
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel
43,howrah,IN,22.5892,88.3103,69,
66,khunti,IN,23.0833,85.2833,51,
246,tanumah,SA,27.1000,44.1333,18,
265,cabo san lucas,MX,22.8909,-109.9124,74,
298,figuig (centre),MA,32.1089,-1.2286,25,
348,san martin,AR,-33.0810,-68.4681,29,
438,vallenar,CL,-28.5708,-70.7581,33,
460,bechar,DZ,31.6167,-2.2167,22,
548,dajal,PK,29.5577,70.3762,34,
604,urangan,AU,-25.3000,152.9000,67,


In [69]:
# Set parameters for hotel search
radius = 10000  
params = {
    'apiKey': geoapify_key,
    'type': 'lodging',
    'categories': 'accommodation.hotel',
    'radius': radius,
}

# Set base URL for finding hotels
base_url = "https://api.geoapify.com/v2/places?"

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    
   # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    # params["filter"] = f"lat:{lat} lon:{lng} radius:{radius}"
    # params["bias"] = f"lat:{lat} lon:{lng}"
    params["lat"] = lat
    params["lon"] = lng   

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    name_address = response.json()

        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        name_of_hotel = name_address["features"][0]["properties"]["name"]
        hotel_df.loc[index, "Hotel Name"] = name_of_hotel
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
hotel_df


howrah - nearest hotel: Sun India Guest House
khunti - nearest hotel: The Raso
tanumah - nearest hotel: فندق الصالح
cabo san lucas - nearest hotel: Comfort Rooms
figuig (centre) - nearest hotel: No hotel found
san martin - nearest hotel: Hotel Lacava
vallenar - nearest hotel: Hotel del Marqués
bechar - nearest hotel: Hôtel Madania
dajal - nearest hotel: City Marriage Hall
urangan - nearest hotel: Ramada


,City,Country,Lat,Lng,Humidity,Hotel,Hotel Name
43,howrah,IN,22.5892,88.3103,69,,Sun India Guest House
66,khunti,IN,23.0833,85.2833,51,,The Raso
246,tanumah,SA,27.1000,44.1333,18,,فندق الصالح
265,cabo san lucas,MX,22.8909,-109.9124,74,,Comfort Rooms
298,figuig (centre),MA,32.1089,-1.2286,25,,No hotel found
348,san martin,AR,-33.0810,-68.4681,29,,Hotel Lacava
438,vallenar,CL,-28.5708,-70.7581,33,,Hotel del Marqués
460,bechar,DZ,31.6167,-2.2167,22,,Hôtel Madania
548,dajal,PK,29.5577,70.3762,34,,City Marriage Hall
604,urangan,AU,-25.3000,152.9000,67,,Ramada


In [97]:
# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(hotel_df, geometry=gpd.points_from_xy(hotel_df['Lng'], hotel_df['Lat']))
gdf['Sizer'] = gdf['Humidity'] ** 0.5

# Create a Points element with hover information
points = gv.Points(gdf, vdims=['Lng', 'Lat', 'City', 'Humidity', 'Sizer', 'Country', 'Hotel Name'])

# Set a reasonable range for color mapping
color_range = (gdf['Sizer'].min(), gdf['Sizer'].max())

# Create the map using the GeoViews map function
result_map = gv.tile_sources.OSM() * points.opts(
    size='Sizer',
    color='City',  
    cmap='Category10',
    colorbar=True,  
    clim=color_range,  
    tools=[HoverTool(tooltips=[('Lng', '@Lng'), ('Lat', '@Lat'), ('City', '@City'), ('Humidity', '@Humidity'), ('Country', '@Country'), ('Hotel', '@Hotel_Name')])]
).opts(
    width=1000,
    height=750,
    legend_position='right',
    legend_offset=(0, 450),  
    alpha=0.7
)

# Display the result
result_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (Lng,Lat,City,Humidity,Sizer,Country,Hotel Name)